## 使用 Pyomo 建模
### ConcreteModel
《Pyomo – Optimization Modeling in Python》一书中 2.3.1 A Concrete Formulation 节关于购买冰淇淋和花生最大化快乐值的例子

\begin{aligned}
&\textbf{Given} && A=\{\text{I\_C\_Scoops},\,\text{Peanuts}\},\\
&&& h_i,\ d_i,\ c_i,\ u_i\ \text{for } i\in A,\ \text{and budget } b.\\[2mm]
&\textbf{Maximize} && 
z \;=\; \sum_{i\in A} h_i\!\left(1 - \frac{u_i}{d_i^{2}}\right)\,x_i \\[1mm]
&\textbf{subject to} &&
\sum_{i\in A} c_i\,x_i \;\le\; b,\\
&&& 0 \;\le\; x_i \;\le\; u_i,\quad \forall i\in A.
\end{aligned}

所使用的数据

\begin{aligned}
&A=\{\text{I\_C\_Scoops},\,\text{Peanuts}\},\quad
b=12.0,\\
&h_{\text{I\_C\_Scoops}}=1,\ h_{\text{Peanuts}}=0.1,\\
&d_{\text{I\_C\_Scoops}}=5,\ d_{\text{Peanuts}}=27,\\
&c_{\text{I\_C\_Scoops}}=3.14,\ c_{\text{Peanuts}}=0.2718,\\
&u_{\text{I\_C\_Scoops}}=100,\ u_{\text{Peanuts}}=40.6.
\end{aligned}


In [9]:
import pyomo.environ as pyo

def IC_model_dict(ICD):
    # ICD is a dictionary with the data for the problem
    
    model = pyo.ConcreteModel(name = "(H)")
    
    model.A = pyo.Set(initialize=ICD["A"])
    
    model.h = pyo.Param(model.A, initialize=ICD["h"])
    model.d = pyo.Param(model.A, initialize=ICD["d"])
    model.c = pyo.Param(model.A, initialize=ICD["c"])
    model.b = pyo.Param(initialize=ICD["b"])
    model.u = pyo.Param(model.A, initialize=ICD["u"])
    
    def xbounds_rule(model, i):
        return (0, model.u[i])
    model.x = pyo.Var(model.A, bounds=xbounds_rule)
    
    def obj_rule(model):
        return sum(model.h[i]*(1 - model.u[i]/model.d[i]**2) * model.x[i] \
            for i in model.A)
    model.z = pyo.Objective(rule=obj_rule,sense=pyo.maximize)
    
    def budget_rule(model):
        return sum(model.c[i]*model.x[i]\
            for i in model.A) <= model.b
    model.budgetconstr = pyo.Constraint(rule=budget_rule)
    
    return model
ICD = {
    "A": ['I_C_Scoops', 'Peanuts'],
    "h": {'I_C_Scoops': 1, 'Peanuts': 0.1},
    "d": {'I_C_Scoops': 5, 'Peanuts': 27},
    "c": {'I_C_Scoops': 3.14, 'Peanuts': 0.2718},
    "b": 12,
    "u": {'I_C_Scoops': 100, 'Peanuts': 40.6}
}

model = IC_model_dict(ICD)

opt = pyo.SolverFactory('glpk')
results = opt.solve(model) # solves and updates model
pyo.assert_optimal_termination(results)

model.display()

Model '(H)'

  Variables:
    x : Size=2, Index=A
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
        I_C_Scoops :     0 :   0.0 :   100 : False : False :  Reals
           Peanuts :     0 :  40.6 :  40.6 : False : False :  Reals

  Objectives:
    z : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 3.8338875171467763

  Constraints:
    budgetconstr : Size=1
        Key  : Lower : Body     : Upper
        None :  None : 11.03508 :  12.0


求解结果

$$
x_{\text{I\_C\_Scoops}}^{\star}=0,\qquad
x_{\text{Peanuts}}^{\star}=40.6.
$$

Budget usage:
$$
\sum_{i\in A} c_i x_i
= 3.14\cdot 0 \;+\; 0.2718\cdot 40.6
= 11.03508 \;\le\; b=12.
$$

Objective value:
$$
z^{\star} = 3.8338875171467763.
$$


### AbstractModel

根据《MATPOWER User's Manual》一书中 6.2 *Standard DC OPF* 节式 (6.27) 至 (6.33) 的描述：

优化变量为  

$$
x = 
\begin{bmatrix}
    \Theta \\
    P_g
\end{bmatrix}
\tag{6.27}
$$

目标函数  

$$
\min_{\Theta, P_g} \; \sum_{i=1}^{n_g} f_i(P_g^i)
\tag{6.28}
$$

约束条件  
有功功率平衡约束
$$
g_P(\Theta, P_g) = B_{\text{bus}} \Theta + P_{\text{bus,shift}} + P_d + G_{\text{sh}} - C_g P_g = 0
\tag{6.29}
$$
支路功率上限约束
$$
h_f(\Theta) = B_f \Theta + P_{f,\text{shift}} - F_{\max} \le 0
\tag{6.30}
$$
$$
h_t(\Theta) = -B_f \Theta - P_{f,\text{shift}} - F_{\max} \le 0
\tag{6.31}
$$
参考节点角度约束
$$
\Theta_i^{\text{ref}} \le \theta_i \le \Theta_i^{\text{ref}}, \quad i \in I_{\text{ref}}
\tag{6.32}
$$
发电机出力上下限约束
$$
p_g^{i,\min} \le p_g^i \le p_g^{i,\max}, \quad i = 1, \dots, n_g
\tag{6.33}
$$

In [10]:
import pyomo.environ as pyo

def build_dc_opf_model():
    """Abstract DC-OPF model (variables: Θ, Pg)
       Obj: min Σ_i f_i(Pg_i) with quadratic costs
       Cons: nodal balance, line limits, reference angles, Pg bounds.
    """
    m = pyo.AbstractModel()

    # ---------- Sets ----------
    m.N = pyo.Set(doc="Buses")                      # buses
    m.G = pyo.Set(doc="Generators")                 # generators
    m.L = pyo.Set(doc="Lines")                      # lines/branches
    m.I_ref = pyo.Set(within=m.N, doc="Reference buses (angle fixed)")

    # ---------- Parameters ----------
    # Quadratic generation costs f_i(Pg_i) = a_i Pg_i^2 + b_i Pg_i + c_i
    m.a = pyo.Param(m.G, within=pyo.NonNegativeReals, doc="Quad cost coeff")
    m.b = pyo.Param(m.G, doc="Linear cost coeff")
    m.c = pyo.Param(m.G, doc="Const cost term")

    # Network matrices and injections
    m.Bbus = pyo.Param(m.N, m.N, doc="Bus susceptance matrix (DC)")
    m.Bf   = pyo.Param(m.L, m.N, doc="Branch flow shift matrix (DC)")
    m.Cg   = pyo.Param(m.N, m.G, within=pyo.NonNegativeReals, doc="Gen-to-bus map")

    m.Pd          = pyo.Param(m.N, doc="Demand at bus (positive load)")
    m.Gsh         = pyo.Param(m.N, default=0.0, doc="Shunt conductance injections")
    m.Pbus_shift  = pyo.Param(m.N, default=0.0, doc="Bus-level phase shift injections")
    m.Pf_shift    = pyo.Param(m.L, default=0.0, doc="Branch-level phase shift injections")
    m.Fmax        = pyo.Param(m.L, within=pyo.NonNegativeReals, doc="Flow limits (abs)")

    # Generator bounds
    m.Pg_min = pyo.Param(m.G, doc="Generator min output")
    m.Pg_max = pyo.Param(m.G, doc="Generator max output")

    # Reference angles
    m.theta_ref = pyo.Param(m.I_ref, default=0.0, doc="Fixed angle value on ref buses")

    # ---------- Variables ----------
    m.theta = pyo.Var(m.N, domain=pyo.Reals, doc="Bus voltage angles (rad)")
    def _pg_bounds(m, g):
        return (m.Pg_min[g], m.Pg_max[g])
    m.Pg = pyo.Var(m.G, bounds=_pg_bounds, doc="Generator outputs (MW)")

    # ---------- Objective: minimize total generation cost ----------
    def total_cost(m):
        return sum(m.a[g]*m.Pg[g]**2 + m.b[g]*m.Pg[g] + m.c[g] for g in m.G)
    m.Obj = pyo.Objective(rule=total_cost, sense=pyo.minimize)

    # ---------- Power balance at each bus ----------
    def power_balance(m, n):
        # Bbus*theta + Pbus_shift + Pd + Gsh - Cg*Pg = 0
        lhs = sum(m.Bbus[n,k]*m.theta[k] for k in m.N) \
              + m.Pbus_shift[n] + m.Pd[n] + m.Gsh[n] \
              - sum(m.Cg[n,g]*m.Pg[g] for g in m.G)
        return lhs == 0
    m.PBalance = pyo.Constraint(m.N, rule=power_balance)

    # ---------- Line flow limits: -Fmax <= Bf*theta + Pf_shift <= Fmax ----------
    def flow_upper(m, l):
        return sum(m.Bf[l,n]*m.theta[n] for n in m.N) + m.Pf_shift[l] <= m.Fmax[l]
    def flow_lower(m, l):
        return -sum(m.Bf[l,n]*m.theta[n] for n in m.N) - m.Pf_shift[l] <= m.Fmax[l]
    m.FlowU = pyo.Constraint(m.L, rule=flow_upper)
    m.FlowL = pyo.Constraint(m.L, rule=flow_lower)

    # ---------- Reference bus angle(s) ----------
    def ref_angle(m, n):
        return m.theta[n] == m.theta_ref[n]
    m.Ref = pyo.Constraint(m.I_ref, rule=ref_angle)

    return m


In [11]:
import numpy as np
from case9 import case9

ppc = case9()
baseMVA = float(ppc["baseMVA"])

bus     = ppc["bus"]
branch  = ppc["branch"]
gen     = ppc["gen"]
gencost = ppc["gencost"]

# --- Sets ---
N = [int(bi[0]) for bi in bus]                 # buses 1..9
G = list(range(1, len(gen)+1))                 # generators labeled 1..3
L = list(range(1, len(branch)+1))              # lines 1..nl

# --- Parameters (per-unit where appropriate) ---
Pd     = {int(bi[0]): float(bi[2])/baseMVA for bi in bus}
Pg_min = {g: float(gen[g-1, 9])/baseMVA for g in G}
Pg_max = {g: float(gen[g-1, 8])/baseMVA for g in G}

# 1) COST COEFFICIENTS: convert to p.u. variable Pg_pu (Pg_MW = baseMVA * Pg_pu)
# f = a*Pg_MW^2 + b*Pg_MW + c = (a*S^2)*Pg_pu^2 + (b*S)*Pg_pu + c
a = {g: float(gencost[g-1, 4]) * (baseMVA**2) for g in G}
b = {g: float(gencost[g-1, 5]) *  baseMVA      for g in G}
c = {g: float(gencost[g-1, 6])                   for g in G}

# 2) LINE LIMITS: treat rateA == 0 as "no limit" (use a very large number in p.u.)
BIG = 1e6
raw_Fmax = [float(branch[l-1, 5]) for l in L]              # MW
Fmax = {l: (raw_Fmax[l-1]/baseMVA if raw_Fmax[l-1] > 0.0 else BIG)
        for l in L}                                        # p.u.

# --- Bus–generator incidence: Cg[n, g] = 1 if gen g located at bus n, else 0 ---
Cg = {(int(bus_i), g): (1 if int(gen[g-1, 0]) == bus_i else 0)
      for bus_i in N for g in G}

# --- Susceptance matrices (DC) ---
nb = len(N)
nl = len(L)
Bf   = np.zeros((nl, nb))
Bbus = np.zeros((nb, nb))

for idx, row in enumerate(branch):
    fbus, tbus, r, x = row[0:4]
    status = row[10]
    if status == 0:
        continue
    suscept = 1.0 / float(x)       # renamed variable
    f = int(fbus) - 1
    t = int(tbus) - 1
    Bf[idx, f]  =  suscept
    Bf[idx, t]  = -suscept
    Bbus[f, f] +=  suscept
    Bbus[t, t] +=  suscept
    Bbus[f, t] += -suscept
    Bbus[t, f] += -suscept

# --- Reference bus ---
I_ref = [1]

# --- Assemble data dict for Pyomo AbstractModel ---
DATA = {
    None: {
        'N': N,
        'G': G,
        'L': L,
        'I_ref': I_ref,
        'Pd': Pd,
        'Pg_min': Pg_min,
        'Pg_max': Pg_max,
        'a': a,
        'b': b,     # still the cost coefficient mapping (not overwritten)
        'c': c,
        'Fmax': Fmax,
        'Cg': Cg,
        # Pyomo wants mapping (i,j)->value with 1-based keys for indexed Param
        'Bbus': {(i+1, j+1): float(Bbus[i, j]) for i in range(nb) for j in range(nb)},
        'Bf': {(l, n): float(Bf[l-1, n-1]) for l in L for n in N},
    }
}

# (Optional) quick sanity checks:
# print(type(DATA[None]['b']), list(DATA[None]['b'].items()))
# print(next(iter(DATA[None]['Bbus'].items())))
# print(next(iter(DATA[None]['Bf'].items())))


In [17]:
import numpy as np
model    = build_dc_opf_model()
instance = model.create_instance(data=DATA)
solver   = pyo.SolverFactory('ipopt')
results  = solver.solve(instance)
pyo.assert_optimal_termination(results)

print("Objective value:", pyo.value(instance.Obj))
for g in instance.G:
    print(f"Pg[{g}] =", pyo.value(instance.Pg[g]))
for n in instance.N:
    theta_rad = pyo.value(instance.theta[n])
    theta_deg = np.rad2deg(theta_rad)
    print(f"θ[{n}] (deg) =", theta_deg)


Objective value: 5216.0266077472725
Pg[1] = 0.8656449793183096
Pg[2] = 1.3437758555814034
Pg[3] = 0.9405791651002866
θ[1] (deg) = -1.4387697021640423e-40
θ[2] (deg) = 6.040180330197329
θ[3] (deg) = 4.00290602235641
θ[4] (deg) = -2.8568335030058063
θ[5] (deg) = -4.635221603243173
θ[6] (deg) = 0.8448807379067652
θ[7] (deg) = -1.3379755517376215
θ[8] (deg) = 1.228137509172601
θ[9] (deg) = -5.429570869918407


In [14]:
# --- Run DCOPF using PYPOWER built-in solver -----------------
from pypower.api import case9, rundcopf, printpf

# Load the case
ppc = case9()

# Run DC-OPF
results = rundcopf(ppc)

# Access structured results
print("\n--- Objective value ---")
print(results['f'])           # total cost in $/h

print("\n--- Generator outputs (MW) ---")
print(results['gen'][:, 1])   # Pg in MW

print("\n--- Bus voltage angles (degrees) ---")
print(results['bus'][:, 8])


PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.02 seconds
Objective Function Value = 5216.03 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0           0.0 to 0.0
Generators         3     On-line Capacity       820.0           0.0 to 0.0
Committed Gens     3     Generation (actual)    315.0               0.0
Loads              3     Load                   315.0               0.0
  Fixed            3       Fixed                315.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           9     Losses (I^2 * Z)         0.0

## MATPOWER CASE 格式与其他格式
### mpc 格式

根据《MATPOWER User's Manual》附录 B （Appendix B Data File Format）中的说明，以及表 B-1 至 B-5 的定义，在本文所使用的 DC-OPF 模型（式 6.27 至 6.33）中，各列向量名称（MATPOWER 中 mpc 风格）与 Pyomo 抽象模型 / 输入数据字典 (DATA) 及其在公式中的符号对应关系如下所示。

| 数学符号 (symbol)        | AbstractModel / DATA 名称  | 字典键名        | mpc 列名（bus/gen/branch） | 中文说明 |
|--------------------------|-----------------------------|------------------|-----------------------------|----------|
| $\Theta$                 | `m.theta[n]`                | `'theta'`        | `Va`（`bus[:,8]`）          | 母线电压角（变量） |
| $P_g$                    | `m.Pg[g]`                   | `'Pg'`           | `Pg`（`gen[:,1]`）          | 发电机有功出力（变量） |
| $f_i(P_g)$ 成本函数项     | `m.a[g], m.b[g], m.c[g]`     | `'a'`, `'b'`, `'c'` | `gencost[:,4:7]`           | 发电成本函数系数 |
| $P_d$                    | `m.Pd[n]`                   | `'Pd'`           | `PD`（`bus[:,2]`）          | 母线负荷有功（参数） |
| $C_g$                    | `m.Cg[n,g]`                 | `'Cg'`           | `gen[:,0] → bus_i`          | 发电机与母线映射矩阵 |
| $B_{\text{bus}}$         | `m.Bbus[n,k]`               | `'Bbus'`         | `branch[:,3]` 反算           | 母线电纳矩阵 |
| $B_f$                    | `m.Bf[l,n]`                 | `'Bf'`           | `branch[:,3]` 反算           | 线路-母线映射电纳矩阵 |
| $F_{\max}$               | `m.Fmax[l]`                 | `'Fmax'`         | `RATE_A`（`branch[:,5]`）    | 线路功率上限 |
| $\theta_{\text{ref}}$    | `m.theta_ref[n]`            | `'theta_ref'`    | `bus_i = 1` 指定             | 参考母线角度（固定值） |
| $p_g^{\min}$             | `m.Pg_min[g]`               | `'Pg_min'`       | `PMIN`（`gen[:,9]`）         | 发电机最小出力 |
| $p_g^{\max}$             | `m.Pg_max[g]`               | `'Pg_max'`       | `PMAX`（`gen[:,8]`）         | 发电机最大出力 |
| $G_{sh}$                 | `m.Gsh[n]`                  | `'Gsh'`          | `GS`（`bus[:,4]`）           | 母线对地电导（并联注入） |
| $P_{\text{bus,shift}}$   | `m.Pbus_shift[n]`           | `'Pbus_shift'`   | `angle`（`branch[:,9]`）→推导 | 相移引起的注入功率项 |
| $P_{f,\text{shift}}$     | `m.Pf_shift[l]`             | `'Pf_shift'`     | `angle`（`branch[:,9]`）→推导 | 相移引起的线路偏置功率 |

 

### 其他格式
参考 IEEE 118 节点系统提供方网站：[Electric Grid Test Case Archive – File Formats](https://electricgrids.engr.tamu.edu/file-formats/) 中的信息

除 MATPOWER / PYPOWER 使用的 **mpc-style** (`case.m` 或 `case.py`) 数据格式外，主流的电力系统模拟软件还包括 PowerWorld Simulator and PowerWorld DS，PSS/E，PSLF。
上述软件各自支持格式如下
#### PowerWorld Simulator and PowerWorld DS
支持的格式包括 `.pwb`, `.pwd`, `.tsb`, `.aux`。并且该软件支持将 case 保存为 PSS/E 或 MATPOWER 采用的格式。
- 完整的电网信息保存在 `.pwb` 中，而 `.pwd` 仅包含单线图。
- `.tsb` 是文本文件格式,包含负荷和发电机每小时数据的二进制时间序列文件。
- `.aux` 文本文件格式。可以包含潮流、动态、故障分析、经济研究等数据。
#### PSS/E
- `.raw` 文本文件，仅包含潮流数据，MATPOWER 提供 `passe2mpc` 进行转换。
- `.dyr` 文本文件，仅包含动态数据。
- `.gic` 文本文件，仅包含地磁感应电流数据。
`.raw` 格式在 MATPOWER 中有 `psse2mpc` 方法转换为 `mpc` 结构的 case. 并在许多 Python 工具库中收到支持，例如`GridCal`。因为 PSS/E 软件本身提供 Python API `psspy`。
#### PSLF
- `.epc` 文本文件仅包含潮流信息。
- `.dyd` 文本文件仅包含动态数据。
#### Pandapower
可以导入或导出 MATPOWER 支持的格式 `pandapower.converter.matpower.from_mpc('case9.m')`。

### 生成训练样本
常用的数据集有：
- 随机生成  
  从 IEEE 等标准案例出发，扰动负荷或发电机出力（按基准功率一定比例 ±20%～30% 随机波动），然后使用求解器计算，所得格式与使用求解器结果格式一致。

- NREL’s OPFLearnData  
  以 **纯文本 `.csv` 文件** 分发，每行表示一个完整的 OPF 输入+输出。字段包括各母线负荷、发电机出力、电压、线路功率流及约束对偶变量。没有专属 API，用户需用自行读取和处理。

- DeepMind’s OPFData  
  每个样本为一个 `.json` 文件，已集成进 **PyTorch Geometric 的 `OPFDataset` 类**，加载即得异构图数据，可直接用于 GNN 训练。

- PowerGraph (NeurIPS 2024 数据集)  
  数据以 `.mat` 格式组织（兼容 MATLAB / Python），官方提供 **PyTorch Geometric 加载器代码库**，支持快速构造 `Data` / `HeteroData` 图结构，支持标准 GNN 模型训练。

## 电力经济调度 DCOPF 模型其他常见约束

### 发电机爬坡速率约束

限制机组在相邻时段出力变化幅度，避免不切实际的突变，确保调度可行性和物理实现。
 来源：[IEEE Std 1547, NERC BAL-002-3](https://unitil.com/sites/default/files/2022-09/Default_IEEE1547-2018_Settings_Requirements_Issued.pdf)

> *“The DER active power output shall increase linearly or in a stepwise linear ramp with a default time of 300s, with steps no greater than 20% of the DER rating. The DER may increase slower than specified...”*
>  *“...within the Contingency Event Recovery Period, demonstrate recovery by returning its Reporting ACE to at least the recovery value of: zero...”*

$$
\begin{aligned}
P_{g,t} - P_{g,t-1} &\le R_g^{\text{up}} \\
P_{g,t-1} - P_{g,t} &\le R_g^{\text{down}}
\end{aligned}
$$

式中，$P_{g,t}$ 和 $P_{g,t-1}$ 分别为机组 $g$ 在时段 $t$ 和 $t-1$ 的出力；$R_g^{\text{up}}$ 为机组 $g$ 的最大爬坡上升速率；$R_g^{\text{down}}$ 为机组 $g$ 的最大爬坡下降速率。

### 备用容量约束

确保在突发负荷或发电机故障时，系统有足够的可调机组快速补偿缺口，保障可靠性。
 来源：[NERC BAL-002-3](https://www.nerc.com/pa/Stand/Reliability%20Standards/BAL-002-3.pdf#:~:text=R2,Time%20Horizon)

> *“Each Responsible Entity shall develop, review and maintain annually, and implement an Operating Process as part of its Operating Plan to determine its Most Severe Single Contingency and make preparations to have Contingency Reserve equal to, or greater than the Responsible Entity’s Most Severe Single Contingency available for maintaining system reliability.”*

$$
\begin{aligned}
R_g^{\text{sp}} &\le P_g^{\max} - P_g \\
\sum_{g} R_g^{\text{sp}} &\ge R^{\text{req}}
\end{aligned}
$$

式中，$R_g^{\text{sp}}$ 为机组 $g$ 提供的备用容量；$P_g^{\max}$ 为机组 $g$ 的最大出力，$P_g$ 为其当前出力（因此 $P_g^{\max} - P_g$ 表示该机组尚未利用的备用裕度）；$R^{\text{req}}$ 为系统要求的总备用容量下限。

### N-1 安全约束

在任一设备（线路或机组）失效情景下系统仍不超限，是调度合规的硬性安全标准。
 来源：[NERC TPL-001-5](https://www.nerc.com/pa/Stand/Reliability%20Standards/TPL-001-5.pdf)

> *“f. Applicable Facility Ratings shall not be exceeded.”*

$$
\begin{aligned}
F_l^{(c)} &\le F_l^{\max}, \quad \forall c \in \text{contingencies} \\
\sum_{g \ne i} P_g^{(c')} &= \text{demand}, \quad \text{if } g_i \text{ fails}
\end{aligned}
$$

式中，$F_l^{(c)}$ 表示在故障情景 $c$ 下线路 $l$ 上的功率潮流，$F_l^{\max}$ 为线路 $l$ 的最大允许输送功率；第二行式子表示当某机组 $g_i$ 故障时，其余机组在该故障情景 $c'$ 下的出力之和应等于系统总需求（demand），确保任一单元失效时（N-1）系统仍能维持功率平衡。

### 分段线性发电成本函数

逼近实际非线性燃料成本或报价曲线，使调度问题适配 LP 或 MILP 求解。
 来源：[PJM 市场规则, MATPOWER 手册](https://matpower.app/manual/matpower/StandardExtensions.html)

> *“The standard OPF formulation ... does not directly handle the non-smooth piecewise linear cost functions that typically arise from discrete bids and offers in electricity markets. When such cost functions are convex, however, they can be modeled using a constrained cost variable (CCV) method.”*

$$
\begin{aligned}
P_g &= \sum_{k=1}^{n} P_{g,k} y_{g,k}, \quad \sum_{k} y_{g,k} = 1 \\
C_g &= \sum_{k=1}^{n} C_{g,k} y_{g,k}, \quad 0 \le y_{g,k} \le 1
\end{aligned}
$$

式中，$P_g$ 为机组 $g$ 的出力；$P_{g,k}$ 和 $C_{g,k}$ 分别表示机组 $g$ 在第 $k$ 个分段对应的功率和该段的成本；$y_{g,k}$ 为机组 $g$ 第 $k$ 段的权重（满足 $\sum_k y_{g,k}=1$，且 $0 \le y_{g,k} \le 1$），用于通过分段插值来确定机组的实际出力和成本。

### 环境排放约束

限制系统总 CO₂或 NOₓ等排放不超过规定配额，推动低碳调度。
 来源：[EU ETS（二氧化碳排放交易制度）](https://icapcarbonaction.com/en/ets-pdf-download/43), [Clean Air Act Amendments 1990](https://users.nber.org/~confer/2004/si2004/ee/gray.pdf)

> *“A cap limits the total emissions allowed in the system. It is set to reduce covered sectors’ emissions by 62% compared to 2005 levels by 2030.”*
>
>  *“Title IV of the 1990 CAAA required the dirtiest coal-fired utilities to cap their SO₂ emissions at 5.8 million tons per year (approximately 33% below their 1990 levels) starting in 1995.”*


$$
\begin{aligned}
e_g &= \alpha_g P_g \\
\sum_{g} e_g &\le E^{\text{max}}
\end{aligned}
$$

式中，$e_g$ 为机组 $g$ 的排放量（如 CO₂ 当量）；$\alpha_g$ 为机组 $g$ 每单位出力的排放系数；$P_g$ 为机组 $g$ 的出力；$E^{\text{max}}$ 为系统允许的总排放配额上限。

### 负荷切除约束

允许在无解或事故场景中通过拉闸一部分负荷来保底供电，最小化系统崩溃风险。
 来源：[NERC EOP-003-2](https://www.nerc.com/pa/Stand/Reliability%20Standards/EOP-003-2.pdf#:~:text=3,uncontrolled%20failure%20of%20the%20Interconnection)

> *“A Balancing Authority and Transmission Operator operating with insufficient generation or transmission capacity must have the capability and authority to shed load rather than risk an uncontrolled failure of the Interconnection.”*

$$
\begin{aligned}
0 \le L_i &\le P_i^{\text{demand}} \\
\sum_i L_i &\le L^{\max}
\end{aligned}
$$

式中，$L_i$ 为节点 $i$ 被切除的负荷量，$P_i^{\text{demand}}$ 为节点 $i$ 的原始需求负荷（因此 $L_i$ 不得超过该值）；$L^{\max}$ 为系统可接受的最大总切负荷量。

### 储能约束

储能设备在不同调度时段间能量变化需满足充放电效率及能量容量限制。
 来源：[IEA 储能报告](https://www.fer.unizg.hr/_download/repository/ES_numerical_examples_v1.0.pdf)

> *“...where variable soeₜ is the current state of energy and soeₜ₋₁ is the previous state of energy for each time-step... Parameters η₍ch₎ and η₍dis₎ are charging and discharging efficiencies of the storage system and Δ is duration of the time-step.”*

$$
\begin{aligned}
b(t+1) &= b(t) + \eta_{\text{ch}} P^{\text{ch}}(t) - \frac{1}{\eta_{\text{dis}}} P^{\text{dis}}(t) \\
0 \le b(t) &\le B^{\max}, \quad R^{\min} \le r(t) \le R^{\max}
\end{aligned}
$$

式中，$b(t)$ 表示储能装置在时段 $t$ 的蓄能量；$\eta_{\text{ch}}$ 和 $\eta_{\text{dis}}$ 分别为储能的充电效率和放电效率；$P^{\text{ch}}(t)$ 和 $P^{\text{dis}}(t)$ 分别表示储能在时段 $t$ 的充电功率和放电功率；$B^{\max}$ 为储能的最大储能容量；$r(t)$ 为储能在时段 $t$ 的净输出功率（正值代表放电，负值代表充电），其取值范围由储能的充放功率下限 $R^{\min}$ 和上限 $R^{\max}$ 决定。